<a href="https://colab.research.google.com/github/HecMartinSol/03MAIR-Alg.Optimizacion/blob/master/AG2/hector_martin_solis_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG2 - Actividad guiada

Héctor Martín Solís

https://colab.research.google.com/drive/1SNDxQQ2oVmF_IqXizcoeuzj9pJ18EO1O

-------- 
## Introducimos el decorador para poder medir tiempos

In [0]:
import time
# DECORADOR PARA MEDIR EL TIEMPO DE LOS ALGORITMOS
def timer_decorator(method):
    def measure_time(*args, **kw):
        time_begin = time.time()
        result = method(*args, **kw)
        time_end = time.time()
        ms_elapsed = (time_end - time_begin) * 1000

        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = ms_elapsed
        else:
            print(method.__name__, "lasted", ms_elapsed, "ms")

        return result
    return measure_time

---------

#### Problema: Viaje por el rio

-  Consideramos una tabla T(i,j) para almacenar todos los precios que nos ofrecen 
los embarcaderos 
- Si no es posible ir desde i a j daremos un valor alto para garantizar que ese
trayecto no se va a elegir en la ruta óptima(modelado habitual para restricciones)
- Establecer una tabla intermedia para de soluciones óptimas parciales para ir
desde i a j.

In [0]:
##OK
tarifas = [
  [0,5,4,3,999,999,999],
  [999,0,999,2,3,999,11],
  [999,999, 0,1,999,4,10],
  [999,999,999, 0,5,6,9],
  [999,999, 999,999,0,999,4],
  [999,999, 999,999,999,0,3],
  [999,999,999,999,999,999,0]
]

In [0]:
def precios(tarifas):
################################################################  
  #Total de Nodos
  n = len(tarifas[0])
  
  #Inicialización de la tabla de precios
  precios = [ [9999] * n  for i in [9999] * n ]
  ruta = [ [""] * n  for i in [""] * n ]
  
  for i in range(n-1):
    for j in range(i+1, n):
      minimo = tarifas[i][j]
      ruta[i][j] = i
      
      for k in range(i, j):
        if precios[i][k] + tarifas[k][j] < minimo:
            minimo = min(minimo, precios[i][k] + tarifas[k][j])
            ruta[i][j] = k

        precios[i][j] = minimo
        
  return precios,ruta

In [75]:
precios,ruta = precios(tarifas)  
#print(precios[0][6])

print("precios")
for i in range(len(tarifas)):
  print(precios[i])

print("rutaS MINIMAS")  
for i in range(len(tarifas)):
  print(ruta[i]) 



precios
[9999, 5, 4, 3, 8, 8, 11]
[9999, 9999, 999, 2, 3, 8, 7]
[9999, 9999, 9999, 1, 6, 4, 7]
[9999, 9999, 9999, 9999, 5, 6, 9]
[9999, 9999, 9999, 9999, 9999, 999, 4]
[9999, 9999, 9999, 9999, 9999, 9999, 3]
[9999, 9999, 9999, 9999, 9999, 9999, 9999]
rutaS MINIMAS
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']


In [76]:
def calcular_ruta(ruta, desde, hasta):
  if desde == hasta:
    return desde 
  else:
    return str(calcular_ruta(ruta, desde, ruta[desde][hasta])) +  ',' + str(ruta[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(ruta, 0,6)


La ruta es:


'0,0,2,5'

----------------------------

#### Problema: Asignación de tareas

- El problema consiste en maximizar el rendimiento en cuanto a la asignación de
N tareas a N agentes. Cada tarea solo puede ser asignado a un agente.
- Los beneficios que se obtienen al realizar la tarea 1 por el agente A es 9
- La matriz de beneficios es la que se muestra en la figura
- Aplicando Ramificación y Poda, obtener la asignación que maximice los
beneficios.

In [0]:
costes = [
  [11,12,18,40],
  [14,15,13,22],
  [11,17,19,23],
  [17,14,20,28],
]

In [78]:
def valor(s, costes):
  valor = 0
  for i in range(len(s)):
    valor += costes[s[i]][i]
  return valor
valor((0,1,2,3), costes)

73

In [0]:
#Coste inferior para soluciones parciales
#  (1,3,) se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def cota_inferior(s,costes):
  valor = 0
  #valores establecidos
  for i in range(len(s)):
    valor += costes[i][s[i]]

  #Estimacion
  for i in range(len(costes)):
    if i not in s:
      valor += min( [ costes[j][i] for j in range(len(s), len(costes))  ])
  return valor

In [0]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(nodo, n):
  hijos = []
  for i in range(n):
    if i not in nodo:
      hijos.append({'s':nodo +(i,)})
  return hijos

In [0]:
def cota_superior(s,costes):
  valor = 0
  #valores establecidos
  for i in range(len(s)):
    valor += costes[i][s[i]]

  #Estimacion
  for i in range(  len(costes)):
    if i not in s:
      valor += max( [ costes[j][i] for j in range(len(s), len(costes))])
  return valor

In [0]:
@timer_decorator
def ramificacion_y_poda(costes):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(costes)
  dimension = len(costes)  
  mejor_solucion=tuple( i for i in range(len(costes)) )
  cot_superior = valor(mejor_solucion,costes)
  #print("Cota Superior:", cot_superior)

  nodos=[]
  nodos.append({'s':(), 'ci':cota_inferior((),costes)    } )

  iteracion = 0

  while( len(nodos) > 0):
    iteracion +=1

    nodo_prometedor = [ min(nodos, key=lambda x:x['ci']) ][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    hijos =[ {'s':x['s'], 'ci':cota_inferior(x['s'], costes)   } for x in crear_hijos(nodo_prometedor, dimension) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    nodo_final = [x for x in hijos if len(x['s']) == dimension  ]
    if len(nodo_final ) >0: 
      #print("\n********Soluciones:",  [x for x in hijos if len(x['s']) == dimension  ] )
      if nodo_final[0]['ci'] < cot_superior:
        cot_superior = nodo_final[0]['ci']
        mejor_solucion = nodo_final
 
    #Poda
    hijos = [x for x in hijos if x['ci'] < cot_superior   ]

    #Añadimos los hijos 
    nodos.extend(hijos) 

    #Eliminamos el nodo ramificado
    nodos =  [  x for x in nodos if x['s'] != nodo_prometedor    ]
   
  print("La solucion final es:" ,mejor_solucion , " en " , iteracion , " iteraciones" , " para dimension: " ,dimension  )

In [89]:
ramificacion_y_poda(costes)

La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  14  iteraciones  para dimension:  4
ramificacion_y_poda lasted 1.6374588012695312 ms


-------
#### SOLUCION POR FUERZA BRUTA
Tiene un coste de O(n!) ya que tiene que explorar las soluciones de todas las permutaciones de agentes y tareas


In [84]:
import itertools

@timer_decorator 
def fuerza_bruta(costes):
  mejor_valor = 10e10
  mejor_solucion = ()

  for s in list(itertools.permutations(range(len(costes)))):
    valor_tmp = valor(s, costes)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s

  print("Mejor solucion:", mejor_solucion, " con valor ", mejor_valor)
fuerza_bruta(costes)

Mejor solucion: (0, 3, 1, 2)  con valor  61
fuerza_bruta lasted 2.1729469299316406 ms


#### COMPROBAR TIEMPOS PARA FUERZA
Realizamos un bucle con matrices aleatoria de costes para colcular cuanto tiempo tarda la ejecución del algoritmo.

A partir de matrices 10x10 los tiempos empiezan a dispararse para el algoritmo de fuerza bruta y empieza a no ser una opción viable

In [85]:
import numpy as np

for i in range(11):
  coste_act = np.random.randint(10, size=(i, i))
  fuerza_bruta(coste_act)


Mejor solucion: ()  con valor  0
fuerza_bruta lasted 1.5757083892822266 ms
Mejor solucion: (0,)  con valor  6
fuerza_bruta lasted 1.5709400177001953 ms
Mejor solucion: (1, 0)  con valor  4
fuerza_bruta lasted 0.19979476928710938 ms
Mejor solucion: (0, 2, 1)  con valor  2
fuerza_bruta lasted 0.18310546875 ms
Mejor solucion: (0, 1, 3, 2)  con valor  17
fuerza_bruta lasted 1.2352466583251953 ms
Mejor solucion: (4, 2, 1, 0, 3)  con valor  19
fuerza_bruta lasted 0.7936954498291016 ms
Mejor solucion: (2, 0, 3, 1, 4, 5)  con valor  8
fuerza_bruta lasted 8.792400360107422 ms
Mejor solucion: (1, 2, 0, 3, 4, 6, 5)  con valor  10
fuerza_bruta lasted 20.39647102355957 ms
Mejor solucion: (5, 0, 1, 2, 3, 7, 4, 6)  con valor  15
fuerza_bruta lasted 177.7360439300537 ms
Mejor solucion: (6, 7, 5, 8, 2, 0, 3, 1, 4)  con valor  11
fuerza_bruta lasted 1750.0693798065186 ms
Mejor solucion: (8, 0, 7, 6, 1, 4, 3, 5, 2, 9)  con valor  6
fuerza_bruta lasted 18878.307342529297 ms


#### COMPROBAR TIEMPOS PARA RAMIFICACION Y PODA
Realizamos un bucle con matrices aleatoria de costes para colcular cuanto tiempo tarda la ejecución del algoritmo.

A partir de matrices 15x15 los tiempos empiezan a dispararse para el algoritmo de fuerza bruta y empieza a no ser una opción viable

In [94]:
import numpy as np

for i in range(18):
  coste_act = np.random.randint(10, size=(i, i))
  ramificacion_y_poda(coste_act)


La solucion final es: ()  en  1  iteraciones  para dimension:  0
ramificacion_y_poda lasted 3.4945011138916016 ms
La solucion final es: (0,)  en  1  iteraciones  para dimension:  1
ramificacion_y_poda lasted 1.1758804321289062 ms
La solucion final es: [{'s': (1, 0), 'ci': 4}]  en  2  iteraciones  para dimension:  2
ramificacion_y_poda lasted 1.2767314910888672 ms
La solucion final es: [{'s': (1, 2, 0), 'ci': 16}]  en  3  iteraciones  para dimension:  3
ramificacion_y_poda lasted 1.184225082397461 ms
La solucion final es: [{'s': (2, 0, 3, 1), 'ci': 7}]  en  10  iteraciones  para dimension:  4
ramificacion_y_poda lasted 1.9583702087402344 ms
La solucion final es: [{'s': (1, 2, 4, 3, 0), 'ci': 15}]  en  21  iteraciones  para dimension:  5
ramificacion_y_poda lasted 1.4007091522216797 ms
La solucion final es: [{'s': (4, 5, 0, 1, 3, 2), 'ci': 9}]  en  26  iteraciones  para dimension:  6
ramificacion_y_poda lasted 2.650022506713867 ms
La solucion final es: [{'s': (0, 4, 2, 5, 1, 3, 6), 'ci':

KeyboardInterrupt: ignored